# [Bonus Tasks](svm_segm.ipynb)

In [34]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.ndimage as ndi
import skimage.morphology as morph
import skimage.util
from sklearn.pipeline import make_pipeline
from skimage.util import view_as_blocks
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [35]:
def blend_result(img, seg, border_radius=1, interior_opacity=1, interior_opacity_decay=0.9, color=(0,1,0)):
    img  = np.dstack([img] * 3).copy()
    img -= img.min()
    img /= img.max()
    selem  = morph.disk(border_radius)
    seg_bd = np.logical_xor(morph.binary_dilation(seg, selem), morph.binary_erosion(seg, selem))
    mask_decay = ndi.distance_transform_edt(seg)
    for i in range(3):
        opacity = interior_opacity / pow(1 + mask_decay[seg], interior_opacity_decay)
        img[:,:,i][seg] = color[i] * opacity + (1 - opacity) * img[:,:,i][seg]
        img[:,:,i][seg_bd] = color[i]
    return img

In [36]:
patch_size = 32

---

**[Task 1.1.]()** Implement `create_data_matrix`:

In [37]:
def create_data_matrix(img):
    patches = view_as_blocks(img, block_shape = (patch_size, patch_size))
    matrix_resh = patches.reshape(-1, patch_size, patch_size)
    
    return matrix_resh


**[Task 1.2.]()** Implement `create_gt_labels_vector`:

In [38]:
def create_gt_labels(gt, patch_size):
    gt.astype(bool)
    
    patches_gt = view_as_blocks(gt, block_shape = (patch_size, patch_size))
    a, b = patches_gt.shape[:2]
    vec_reshaped = patches_gt.reshape(a * b, -1)
    fg = vec_reshaped.mean(axis = 1)
    labs = np.where(fg > 0.5,1,np.where(fg == 0, -1, 0))

**[Task 1.3.]()** Create the SVM classifier:

In [39]:
clf = make_pipeline(StandardScaler(), SVC(class_weight='balanced', gamma=0.1))

**[Task 1.3 (a).]()** Create the data matrices for the images `dna-33` and `dna-44`:

In [40]:
dna_33 = plt.imread('data/NIH3T3/im/dna-33.png')
dna_44 = plt.imread('data/NIH3T3/im/dna-44.png')

gt_33 = plt.imread('data/NIH3T3/gt/33.png')
gt_44 = plt.imread('data/NIH3T3/gt/44.png')

dm_33 = create_data_matrix(dna_33)
dm_44 = create_data_matrix(dna_44)
gtlab_33 = create_gt_labels(gt_33, patch_size_xy)
gtlab_44 = create_gt_labels(gt_44, patch_size_xy)

**[Task 1.3 (b).]()** Create the corresponding ground truth label vectors:

In [42]:
print(dm_33)

[[[0.15686275 0.15686275 0.15294118 ... 0.13725491 0.13333334 0.13725491]
  [0.16078432 0.15294118 0.14509805 ... 0.14509805 0.13725491 0.13725491]
  [0.14901961 0.14901961 0.14509805 ... 0.13725491 0.13333334 0.13725491]
  ...
  [0.10980392 0.10980392 0.10588235 ... 0.09019608 0.09411765 0.09411765]
  [0.10196079 0.10196079 0.10588235 ... 0.09019608 0.09019608 0.09019608]
  [0.10980392 0.10196079 0.10588235 ... 0.09019608 0.09019608 0.09019608]]

 [[0.13725491 0.13725491 0.12941177 ... 0.08235294 0.09411765 0.09019608]
  [0.13333334 0.13333334 0.13333334 ... 0.08627451 0.08627451 0.08627451]
  [0.13333334 0.12941177 0.1254902  ... 0.09019608 0.09019608 0.08627451]
  ...
  [0.09019608 0.09019608 0.08627451 ... 0.08627451 0.09019608 0.08235294]
  [0.09019608 0.09019608 0.09019608 ... 0.09019608 0.08627451 0.08627451]
  [0.09411765 0.09019608 0.09019608 ... 0.09019608 0.09411765 0.08627451]]

 [[0.08627451 0.08235294 0.08627451 ... 0.08627451 0.07843138 0.08627451]
  [0.09019608 0.090196

**[Task 1.3 (c).]()** Create the *combined* data matrices and ground truth label vectors:

**[Task 1.3 (d).]()** Train the classifier using the data matrix and label vectors from above:

**[Task 1.4.]()** Implement the function `predict_image`:

Test your implementation:

In [41]:
img = plt.imread(f'data/NIH3T3/im/dna-0.png')
seg = predict_image(img)

plt.figure()
plt.imshow(blend_result(img, seg))

NameError: name 'predict_image' is not defined

**[Task 1.5.]()** Perform batch processing: